In [1]:
!wget https://github.com/niazwazir/Google-Colab/raw/refs/heads/master/WAZIR_GROUND_TRUTH_DATASET.zip

--2025-01-19 21:55:50--  https://github.com/niazwazir/Google-Colab/raw/refs/heads/master/WAZIR_GROUND_TRUTH_DATASET.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/niazwazir/Google-Colab/refs/heads/master/WAZIR_GROUND_TRUTH_DATASET.zip [following]
--2025-01-19 21:55:51--  https://raw.githubusercontent.com/niazwazir/Google-Colab/refs/heads/master/WAZIR_GROUND_TRUTH_DATASET.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18145556 (17M) [application/zip]
Saving to: ‘WAZIR_GROUND_TRUTH_DATASET.zip’

WAZIR_GROUND_TRUTH_ 100%[===================>]  17.30M  --.-KB/s    in 0.1s    

2025-01-

In [2]:
!ls

sample_data  WAZIR_GROUND_TRUTH_DATASET.zip


In [3]:
!unzip WAZIR_GROUND_TRUTH_DATASET.zip

Archive:  WAZIR_GROUND_TRUTH_DATASET.zip
   creating: WAZIR_GROUND_TRUTH_DATASET/
   creating: WAZIR_GROUND_TRUTH_DATASET/SET14/
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/baboon.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/barbara.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/bridge.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/coastguard.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/comic.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/face.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/flowers.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/foreman.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/lenna.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/man.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/monarch.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/pepper.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/ppt3.bmp  
  inflating: WAZIR_GROUND_TRUTH_DATASET/SET14/zebra.bmp  
   creating: WAZIR_GROUND_TRUTH_DATASET/SET5/
 

In [4]:
!ls

sample_data  WAZIR_GROUND_TRUTH_DATASET  WAZIR_GROUND_TRUTH_DATASET.zip


In [5]:
import os
import cv2
import glob  # Import the glob module
import numpy as np

In [6]:

def prepare_test_data_hr_lr():
    path_original = 'WAZIR_GROUND_TRUTH_DATASET'
    datasets = ['Set5', 'Set14', 'T91']
    extensions = ['*.jpg', '*.png', '*.bmp']

    degradation = 'BI'  # BI, BD, DN
    scale_all = [2, 3, 4, 8, 16] if degradation == 'BI' else [3]

    for dataset in datasets:
        print(f'Processing {dataset}:')
        filepaths = []

        # Collect all file paths based on extensions
        for ext in extensions:
            filepaths += glob.glob(os.path.join(path_original, dataset, ext))

        for idx_im, filepath in enumerate(filepaths):
            name_im = os.path.basename(filepath)
            print(f'{idx_im + 1}. {name_im}: ', end='')

            im_ori = cv2.imread(filepath)
            if im_ori.shape[2] == 1:  # Check if image is grayscale
                im_ori = cv2.cvtColor(im_ori, cv2.COLOR_GRAY2BGR)

            for scale in scale_all:
                print(f'x{scale} ', end='')

                im_HR = modcrop(im_ori, scale)

                if degradation == 'BI':
                    im_LR = cv2.resize(im_HR, (0, 0), fx=1/scale, fy=1/scale, interpolation=cv2.INTER_CUBIC)
                elif degradation == 'BD':
                    im_LR = imresize_bd(im_HR, scale, 'Gaussian', sigma=1.6)  # sigma=1.6
                elif degradation == 'DN':
                    np.random.seed(0)  # Fix seed for test data
                    im_LR = imresize_dn(im_HR, scale, sigma=30)  # noise level sigma=30

                # Create folders for HR and LR images
                folder_HR = os.path.join('./HR', dataset, f'x{scale}')
                folder_LR = os.path.join('./LR/LR{degradation}', dataset, f'x{scale}')
                os.makedirs(folder_HR, exist_ok=True)
                os.makedirs(folder_LR, exist_ok=True)

                # Save images
                fn_HR = os.path.join(folder_HR, f'{name_im[:-4]}_HR_x{scale}.png')
                fn_LR = os.path.join(folder_LR, f'{name_im[:-4]}_LR{degradation}_x{scale}.png')

                cv2.imwrite(fn_HR, im_HR)
                cv2.imwrite(fn_LR, im_LR)

            print()
        print()

def modcrop(imgs, modulo):
    if imgs.ndim == 3:  # Color image
        sz = imgs.shape[:2]
        sz -= np.mod(sz, modulo)
        return imgs[:sz[0], :sz[1], :]
    else:  # Grayscale image
        sz = imgs.shape
        sz -= np.mod(sz, modulo)
        return imgs[:sz[0], :sz[1]]

def imresize_bd(im, scale, type_, sigma):
    if type_ == 'Gaussian':
        if scale % 2 == 1:
            kernelsize = int(np.ceil(sigma * 3)) * 2 + 1
            kernel = cv2.getGaussianKernel(kernelsize, sigma)
            kernel = kernel @ kernel.T  # Create a Gaussian kernel
            blur_HR = cv2.filter2D(im, -1, kernel)

            return blur_HR[scale - 1::scale - 1, scale - 1::scale - 1]

        elif scale % 2 == 0:
            kernelsize = int(np.ceil(sigma * 3)) * 2 + 2
            kernel = cv2.getGaussianKernel(kernelsize, sigma)
            kernel = kernel @ kernel.T
            blur_HR = cv2.filter2D(im, -1, kernel)

            return blur_HR[scale // 2::scale][:blur_HR.shape[0] - (scale // 2), scale // 2::scale][:blur_HR.shape[1] - (scale // 2)]

    return cv2.resize(im, (0, 0), fx=1/scale, fy=1/scale)

def imresize_dn(ImHR, scale, sigma):
    ImDown = cv2.resize(ImHR, (0, 0), fx=1/scale, fy=1/scale)
    ImDownNoise = ImDown.astype(np.float32) + sigma * np.random.randn(*ImDown.shape).astype(np.float32)
    return np.clip(ImDownNoise.astype(np.uint8), 0, 255)

# Call the function to prepare the test data
prepare_test_data_hr_lr()

Processing Set5:

Processing Set14:

Processing T91:
1. t38.png: x2 x3 x4 x8 x16 
2. t51.png: x2 x3 x4 x8 x16 
3. t24.png: x2 x3 x4 x8 x16 
4. t21.png: x2 x3 x4 x8 x16 
5. t34.png: x2 x3 x4 x8 x16 
6. tt7.png: x2 x3 x4 x8 x16 
7. t35.png: x2 x3 x4 x8 x16 
8. t11.png: x2 x3 x4 x8 x16 
9. t14.png: x2 x3 x4 x8 x16 
10. t57.png: x2 x3 x4 x8 x16 
11. t18.png: x2 x3 x4 x8 x16 
12. tt2.png: x2 x3 x4 x8 x16 
13. tt13.png: x2 x3 x4 x8 x16 
14. tt3.png: x2 x3 x4 x8 x16 
15. t9.png: x2 x3 x4 x8 x16 
16. t33.png: x2 x3 x4 x8 x16 
17. t56.png: x2 x3 x4 x8 x16 
18. t66.png: x2 x3 x4 x8 x16 
19. t23.png: x2 x3 x4 x8 x16 
20. t31.png: x2 x3 x4 x8 x16 
21. tt16.png: x2 x3 x4 x8 x16 
22. t60.png: x2 x3 x4 x8 x16 
23. tt26.png: x2 x3 x4 x8 x16 
24. tt23.png: x2 x3 x4 x8 x16 
25. t6.png: x2 x3 x4 x8 x16 
26. t48.png: x2 x3 x4 x8 x16 
27. t62.png: x2 x3 x4 x8 x16 
28. t1.png: x2 x3 x4 x8 x16 
29. t22.png: x2 x3 x4 x8 x16 
30. t58.png: x2 x3 x4 x8 x16 
31. t25.png: x2 x3 x4 x8 x16 
32. t50.png: x2 x3 x4 x8 